In [27]:
main_cols=c("USER_ID","YOB","Gender","Income","HouseholdStatus","EducationLevel","Party")

In [164]:
TrainImp = read.csv("pollTrain_imputed.csv")

In [166]:
table(TrainImp$YOB)


1880 1881 1896 1900 1901 1928 1931 1933 1935 1936 1937 1938 1939 1940 1941 1942 
   1    1    1    4    1    2    1    4    2    5    2    8    7   13   15   18 
1943 1944 1945 1946 1947 1948 1949 1950 1951 1952 1953 1954 1955 1956 1957 1958 
  17   20   20   23   31   40   30   41   37   37   36   30   43   47   43   47 
1959 1960 1961 1962 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 1974 
  52   48   54   63   55   62   61   65   69   94   84  117   93  108   86   98 
1975 1976 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 
  98  104  110   87  116  136  131  118  110  111  134  124  152  145  119  174 
1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2003 2011 2013 2039 
 122  157  172  202  285  276  218  183   97   38    8    1    1    1    2 

In [178]:
otrain1 = read.csv("train2016.csv")
nrow(otrain1)

trlab = subset(otrain1$Party,!otrain1$YOB %in% c("2011","2013","2039","1880","1881","1896"))
length(trlab)

[1] 5568

[1] 5561

In [29]:
TrainImp$Party = ifelse(TrainImp$Party=="Democrat",1,0)
table(TrainImp$Party)


   0    1 
2617 2951 

In [30]:
t=c()
t
for (col in setdiff(names(TrainImp),c("USER_ID","Party"))){
  #if mean(is.na(pollTrain[,col])
    cat(col,"cor:",cor(as.numeric(TrainImp[,col]),as.numeric(TrainImp$Party)),"\n")
    val = cor(as.numeric(TrainImp[,col]),as.numeric(TrainImp$Party))
    t = c(t,c(col=val))
    
  #print(paste("The year is", year))
}
names(t) = setdiff(names(TrainImp),c("USER_ID","Party"))

NULL

YOB cor: -0.01367359 
Gender cor: -0.1086779 
Income cor: -0.009572352 
HouseholdStatus cor: -0.009958228 
EducationLevel cor: 0.03703605 
Q124742 cor: -0.008830977 
Q124122 cor: 0.01242852 
Q123464 cor: -0.01076514 
Q123621 cor: -0.02126713 
Q122769 cor: -0.00353109 
Q122770 cor: -0.01845403 
Q122771 cor: 0.05248717 
Q122120 cor: -0.009991358 
Q121699 cor: 0.04793204 
Q121700 cor: 0.03235938 
Q120978 cor: 0.01267589 
Q121011 cor: 0.03004657 
Q120379 cor: 0.05930057 
Q120650 cor: -0.009312594 
Q120472 cor: -0.05475901 
Q120194 cor: -0.02854448 
Q120012 cor: 0.03511101 
Q120014 cor: -0.03205501 
Q119334 cor: 0.00373549 
Q119851 cor: 0.04069271 
Q119650 cor: 0.01673535 
Q118892 cor: 0.03792097 
Q118117 cor: -0.02412698 
Q118232 cor: -0.02633558 
Q118233 cor: -0.0123694 
Q118237 cor: 0.02351877 
Q117186 cor: -0.01388236 
Q117193 cor: -0.003145904 
Q116797 cor: 0.003528791 
Q116881 cor: -0.06630518 
Q116953 cor: 0.02124064 
Q116601 cor: 0.01601747 
Q116441 cor: -0.01068603 
Q116448 cor: 0.

In [32]:
t1=which(abs(t)>0.09)

In [33]:
names(t1)

[1] "Gender"  "Q115611" "Q113181" "Q109244" "Q98197"

In [34]:
TrainImp$Party = as.factor(TrainImp$Party)

In [35]:
library(caTools)
split = sample.split(TrainImp$Party, SplitRatio = 0.75)
Train = subset(TrainImp, split == TRUE)
Test = subset(TrainImp, split == FALSE)

In [36]:
library(rpart)
library(e1071)
library(randomForest)

In [40]:
rfModel1 = randomForest(Party~ Gender+Q115611+Q113181+Q109244+Q98197, data = Train)#, ntree = 1000, sampsize = 300)

In [41]:
predTrain = predict(rfModel1)

In [42]:
t = table(Train$Party,predTrain)
t

   predTrain
       0    1
  0 1144  819
  1  771 1442

In [43]:
sum(diag(t))/sum(t)

[1] 0.6192529

In [44]:
predTest = predict(rfModel1,newdata = Test)

In [45]:
t = table(Test$Party,predTest)
t

   predTest
      0   1
  0 402 252
  1 285 453

In [46]:
sum(diag(t))/sum(t)

[1] 0.6142241

 ntree = 100, nodesize = 50, 0.625718390804598

In [158]:
LogModel = glm(Party~YOB+Gender+HouseholdStatus+EducationLevel+Q98197+Q98078+
               Q98869+Q99480+Q100689+Q101163+Q105840+Q107869+Q109244, data = Train, family=binomial)

In [159]:
summary(LogModel)


Call:
glm(formula = Party ~ YOB + Gender + HouseholdStatus + EducationLevel + 
    Q98197 + Q98078 + Q98869 + Q99480 + Q100689 + Q101163 + Q105840 + 
    Q107869 + Q109244, family = binomial, data = Train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.1417  -1.0885   0.6546   1.1089   1.7618  

Coefficients:
                                           Estimate Std. Error z value Pr(>|z|)
(Intercept)                               15.284091   5.997615   2.548 0.010823
YOB                                       -0.007455   0.003020  -2.468 0.013570
GenderMale                                -0.172025   0.075116  -2.290 0.022015
HouseholdStatusDomestic Partners (w/kids) -0.598289   0.342992  -1.744 0.081102
HouseholdStatusMarried (no kids)          -0.720701   0.217664  -3.311 0.000929
HouseholdStatusMarried (w/kids)           -0.919149   0.207180  -4.436 9.14e-06
HouseholdStatusSingle (no kids)           -0.573698   0.202752  -2.830 0.004661
HouseholdStatusSingle (w

In [160]:
predLogTrain = predict(LogModel,type = "response")

In [161]:
t = table(Train$Party,predLogTrain>0.5)
t
sum(diag(t))/sum(t)

   
    FALSE TRUE
  0  1236  727
  1   856 1357

[1] 0.6209291

In [162]:
predLogTest = predict(LogModel,newdata = Test,type="response")
t = table(Test$Party,predLogTest>0.5)
t
sum(diag(t))/sum(t)

   
    FALSE TRUE
  0   400  254
  1   314  424

[1] 0.591954

In [163]:
Tree1 = rpart(Party~YOB+Gender+Income+HouseholdStatus+EducationLevel+Q122771+Q121699+
              Q120379+Q120472+Q119851+Q116881+Q115611+Q115899+Q113181+Q110740+Q109244+
              Q107869+Q106272+Q106042+Q105840+Q102089+Q101163+Q101596+
               Q100689+Q100680+Q99716+Q99480+Q98869+Q98078+Q98197, data = Train, method="class")

In [168]:
predtreeTest = predict(Tree1,newdata = Test, type = "class")
t = table(Test$Party,predtreeTest)
t
sum(diag(t))/sum(t)

   predtreeTest
      0   1
  0 398 256
  1 264 474

[1] 0.6264368

In [212]:
library(e1071)
library(caret)
numFolds = trainControl(method = "cv",number = 10)
cpGrid = expand.grid(.cp=seq(0.001,0.01,0.001))
summary(cpGrid)

      .cp         
 Min.   :0.00100  
 1st Qu.:0.00325  
 Median :0.00550  
 Mean   :0.00550  
 3rd Qu.:0.00775  
 Max.   :0.01000  

In [213]:
train(Party~ YOB+Gender+Income+HouseholdStatus+EducationLevel+Q122771+Q121699+Q120379+Q120472+Q119851+Q116881+Q115611+Q115899+Q113181+Q110740+Q109244+Q107869+Q106272+Q106042+Q105840+Q102089+Q101163+Q101596+
               Q100689+Q100680+Q99716+Q99480+Q98869+Q98078+Q98197,data = Train, method = "rpart", trControl = numFolds, tuneGrid = cpGrid)

CART 

4176 samples
 107 predictors
   2 classes: '0', '1' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 3757, 3759, 3758, 3758, 3759, 3759, ... 
Resampling results across tuning parameters:

  cp     Accuracy   Kappa      Accuracy SD  Kappa SD  
  0.001  0.5948202  0.1850309  0.02095196   0.04106294
  0.002  0.6094324  0.2124329  0.02213074   0.04440824
  0.003  0.6075214  0.2078185  0.02142081   0.04383429
  0.004  0.6183122  0.2304300  0.02887626   0.05655684
  0.005  0.6180764  0.2299932  0.02984652   0.05864051
  0.006  0.6183156  0.2297567  0.02939771   0.05795685
  0.007  0.6195106  0.2344838  0.02851419   0.05656812
  0.008  0.6223860  0.2417913  0.02861469   0.05670796
  0.009  0.6223860  0.2417913  0.02861469   0.05670796
  0.010  0.6223860  0.2417913  0.02861469   0.05670796

Accuracy was used to select the optimal model using  the largest value.
The final value used for the model was cp = 0.01. 

In [214]:
TestFinal = read.csv("pollTest_imputed.csv")

In [217]:
PredTreeTestFin = predict(Tree1,newdata = TestFinal,type = "class")
PredTreeTestFin = as.factor(ifelse(PredTreeTestFin==1, "Democrat", "Republican"))
table(PredTreeTestFin)
MySubmission = data.frame(USER_ID = TestFinal$USER_ID, Predictions = PredTreeTestFin)
str(MySubmission)
write.csv(MySubmission, "SubmissionRPART_IMP_FE_01.csv", row.names=FALSE)

PredTreeTestFin
  Democrat Republican 
       726        666 

'data.frame':	1392 obs. of  2 variables:
 $ USER_ID    : int  2 3 6 7 14 28 29 37 44 56 ...
 $ Predictions: Factor w/ 2 levels "Democrat","Republican": 1 2 2 1 1 2 1 2 1 1 ...


In [382]:
Train2 = Train[,c("YOB","Gender","Income","HouseholdStatus","EducationLevel")]
Test2 = Test[,c("YOB","Gender","Income","HouseholdStatus","EducationLevel")]

In [383]:
setdiff(names(Train2),names(Test2))

character(0)

In [384]:
for (col in setdiff(names(Train2),c("Party"))){
  #if mean(is.na(pollTrain[,col])
    Train2[,col] = as.numeric(Train2[,col])
    Test2[,col] = as.numeric(Test2[,col])


    
  #print(paste("The year is", year))
}

In [385]:
str(Train2)

'data.frame':	4176 obs. of  5 variables:
 $ YOB            : num  1983 1984 1997 1983 1996 ...
 $ Gender         : num  2 1 1 2 2 2 2 2 2 2 ...
 $ Income         : num  1 3 5 2 4 6 5 4 4 5 ...
 $ HouseholdStatus: num  4 4 5 3 5 5 3 5 4 4 ...
 $ EducationLevel : num  2 6 3 4 3 3 2 3 2 2 ...


In [386]:

preproc = preProcess(Train2)

In [421]:
normTrain = predict(preproc, Train2)
normTest = predict(preproc, Test2)

In [422]:
str(normTest)

'data.frame':	1392 obs. of  5 variables:
 $ YOB            : num  -2.7163 -0.6308 1.1288 0.0861 -0.1746 ...
 $ Gender         : num  0.809 -1.237 0.809 -1.237 -1.237 ...
 $ Income         : num  0.275 0.858 0.275 -0.308 -1.475 ...
 $ HouseholdStatus: num  -0.291 -2.227 0.677 -0.291 -3.195 ...
 $ EducationLevel : num  -0.908 -0.908 1.136 -0.908 -0.908 ...


In [423]:
km = kmeans(normTrain, centers = 2)

In [424]:
table(km$cluster)


   1    2 
2222 1954 

In [425]:
library(flexclust)

In [426]:
km.kcca = as.kcca(km,normTrain)

In [427]:
clusterTrain = predict(km.kcca)
clusterTest = predict(km.kcca, newdata = normTest)

In [428]:
table(clusterTest)

clusterTest
  1   2 
755 637 

In [429]:
length(clusterTest)

[1] 1392

In [430]:
length(clusterTrain)

[1] 4176

In [602]:
cTrain1 = subset(Train, clusterTrain == 1)
cTrain2 = subset(Train, clusterTrain == 2)


In [432]:
names(cTrain1)

[1] "USER_ID"         "YOB"             "Gender"          "Income"         
  [5] "HouseholdStatus" "EducationLevel"  "Party"           "Q124742"        
  [9] "Q124122"         "Q123464"         "Q123621"         "Q122769"        
 [13] "Q122770"         "Q122771"         "Q122120"         "Q121699"        
 [17] "Q121700"         "Q120978"         "Q121011"         "Q120379"        
 [21] "Q120650"         "Q120472"         "Q120194"         "Q120012"        
 [25] "Q120014"         "Q119334"         "Q119851"         "Q119650"        
 [29] "Q118892"         "Q118117"         "Q118232"         "Q118233"        
 [33] "Q118237"         "Q117186"         "Q117193"         "Q116797"        
 [37] "Q116881"         "Q116953"         "Q116601"         "Q116441"        
 [41] "Q116448"         "Q116197"         "Q115602"         "Q115777"        
 [45] "Q115610"         "Q115611"         "Q115899"         "Q115390"        
 [49] "Q114961"         "Q114748"         "Q115195"         "Q114517"        
 [53] "Q114386"         "Q113992"         "Q114152"         "Q113583"        
 [57] "Q113584"         "Q113181"         "Q112478"         "Q112512"        
 [61] "Q112270"         "Q111848"         "Q111580"         "Q111220"        
 [65] "Q110740"         "Q109367"         "Q108950"         "Q109244"        
 [69] "Q108855"         "Q108617"         "Q108856"         "Q108754"        
 [73] "Q108342"         "Q108343"         "Q107869"         "Q107491"        
 [77] "Q106993"         "Q106997"         "Q106272"         "Q106388"        
 [81] "Q106389"         "Q106042"         "Q105840"         "Q105655"        
 [85] "Q104996"         "Q103293"         "Q102906"         "Q102674"        
 [89] "Q102687"         "Q102289"         "Q102089"         "Q101162"        
 [93] "Q101163"         "Q101596"         "Q100689"         "Q100680"        
 [97] "Q100562"         "Q99982"          "Q100010"         "Q99716"         
[101] "Q99581"          "Q99480"          "Q98869"          "Q98578"         
[105] "Q98059"          "Q98078"          "Q98197"          "Q96024"

In [593]:
cTest1 = subset(Test, clusterTest == 1)
cTest2 = subset(Test, clusterTest == 2)

In [549]:
cTree1 = rpart(Party~YOB+Gender+Income+HouseholdStatus+EducationLevel+Q122771+Q121699+Q120379+Q120472+Q119851+Q116881+Q115611+Q115899+Q113181+Q110740+Q109244+Q107869+Q106272+Q106042+Q105840+Q102089+Q101163+Q101596+
               Q100689+Q100680+Q99716+Q99480+Q98869+Q98078+Q98197, data=cTrain1, method = "class",cp=0.006)

In [550]:
t= table(cTrain1$Party,predict(cTree1,type="class"))
t
sum(diag(t))/sum(t)

   
      0   1
  0 662 369
  1 448 743

[1] 0.6323132

In [551]:
t= table(cTest1$Party,predict(cTree1,newdata = cTest1,type="class"))
t
sum(diag(t))/sum(t)

   
      0   1
  0 223 134
  1 155 243

[1] 0.6172185

In [567]:
cTree2 = rpart(Party~YOB+Gender+Income+HouseholdStatus+EducationLevel+Q122771+Q121699+Q120379+Q120472+Q119851+Q116881+Q115611+Q115899+Q113181+Q110740+Q109244+Q107869+Q106272+Q106042+Q105840+Q102089+Q101163+Q101596+
               Q100689+Q100680+Q99716+Q99480+Q98869+Q98078+Q98197, data=cTrain2, method = "class")

In [568]:
t= table(cTrain2$Party,predict(cTree2,type="class"))
t
sum(diag(t))/sum(t)

   
      0   1
  0 617 315
  1 390 632

[1] 0.6392016

In [569]:
t= table(cTest2$Party,predict(cTree2,newdata = cTest2,type="class"))
t
sum(diag(t))/sum(t)

   
      0   1
  0 192 105
  1 132 208

[1] 0.6279435

In [502]:
cLog1 = glm(Party~YOB+Gender+Income+HouseholdStatus+EducationLevel+Q122771+Q121699+Q120379+Q120472+Q119851+Q116881+Q115611+Q115899+Q113181+Q110740+Q109244+Q107869+Q106272+Q106042+Q105840+Q102089+Q101163+Q101596+
               Q100689+Q100680+Q99716+Q99480+Q98869+Q98078+Q98197, data=cTrain1, family=binomial)

In [503]:
t= table(cTrain1$Party,predict(cLog1,type="response")>0.5)
t
sum(diag(t))/sum(t)

   
    FALSE TRUE
  0   549  482
  1   357  834

[1] 0.6224122

In [504]:
t= table(cTest1$Party,predict(cLog1,newdata = cTest1,type="response")>0.5)
t
sum(diag(t))/sum(t)

   
    FALSE TRUE
  0   198  159
  1   134  264

[1] 0.6119205

In [570]:
cLog2 = glm(Party~YOB+Gender+Income+HouseholdStatus+EducationLevel+Q122771+Q121699+Q120379+Q120472+Q119851+Q116881+Q115611+Q115899+Q113181+Q110740+Q109244+Q107869+Q106272+Q106042+Q105840+Q102089+Q101163+Q101596+
               Q100689+Q100680+Q99716+Q99480+Q98869+Q98078+Q98197, data=cTrain2, family=binomial)

In [571]:
t= table(cTrain2$Party,predict(cLog2,type="response")>0.5)
t
sum(diag(t))/sum(t)

   
    FALSE TRUE
  0   609  323
  1   369  653

[1] 0.6458547

In [574]:
t= table(cTest2$Party,predict(cLog2,newdata = cTest2,type="response")>0.6)
t
sum(diag(t))/sum(t)

   
    FALSE TRUE
  0   235   62
  1   186  154

[1] 0.610675

In [578]:
library(class)

In [582]:
knn(cTrain1,cTest1,cTrain1$Party)

Warning message:
In knn(cTrain1, cTest1, cTrain1$Party): NAs introduced by coercionWarning message:
In knn(cTrain1, cTest1, cTrain1$Party): NAs introduced by coercion

ERROR: Error in knn(cTrain1, cTest1, cTrain1$Party): NA/NaN/Inf in foreign function call (arg 6)


In [16]:
Train3 = Train
Test3 = Test

tr_lab= Train3$Party
ts_lab = Test3$Party

Train3$USER_ID = NULL
Test3$USER_ID = NULL
Train3$Party =NULL
Test3$Party = NULL

library(caret)
library(class)

In [17]:
for (col in setdiff(names(Train3),c("Party"))){
  #if mean(is.na(pollTrain[,col])
    Train3[,col] = as.numeric(Train3[,col])
    Test3[,col] = as.numeric(Test3[,col])


    
  #print(paste("The year is", year))
}

preproc1 = preProcess(Train3)
Train4 = predict(preproc1,Train3)
Test4= predict(preproc1,Test3)

In [18]:
str(Train4)

'data.frame':	4176 obs. of  106 variables:
 $ YOB            : num  -2.688 -0.626 0.276 1.113 1.049 ...
 $ Gender         : num  0.796 -1.257 -1.257 -1.257 0.796 ...
 $ Income         : num  0.272 0.85 -0.306 0.85 0.272 ...
 $ HouseholdStatus: num  -0.297 -2.243 -0.297 0.676 0.676 ...
 $ EducationLevel : num  -0.918 -0.918 1.135 -0.405 -0.405 ...
 $ Q124742        : num  -0.799 -0.799 -0.799 -0.799 1.252 ...
 $ Q124122        : num  -1.15 0.869 0.869 -1.15 0.869 ...
 $ Q123464        : num  -0.307 -0.307 -0.307 -0.307 -0.307 ...
 $ Q123621        : num  -0.993 -0.993 -0.993 -0.993 -0.993 ...
 $ Q122769        : num  -0.789 -0.789 -0.789 -0.789 -0.789 ...
 $ Q122770        : num  0.853 0.853 0.853 0.853 0.853 ...
 $ Q122771        : num  0.481 0.481 0.481 0.481 -2.078 ...
 $ Q122120        : num  -0.601 -0.601 -0.601 -0.601 1.665 ...
 $ Q121699        : num  0.619 0.619 0.619 -1.614 -1.614 ...
 $ Q121700        : num  -0.443 -0.443 -0.443 -0.443 -0.443 ...
 $ Q120978        : num  -1.12

In [25]:
PredKnn=knn(Train4,Test4,tr_lab,k=40)

In [26]:
t=table(ts_lab,PredKnn)
t
sum(diag(t))/sum(t)

      PredKnn
ts_lab   0   1
     0 354 300
     1 252 486

[1] 0.6034483

In [629]:
?knn

knn {class},R Documentation
train,matrix or data frame of training set cases.
test,matrix or data frame of test set cases. A vector will be interpreted as a row vector for a single case.
cl,factor of true classifications of training set
k,number of neighbours considered.
l,"minimum vote for definite decision, otherwise doubt. (More precisely, less than k-l dissenting votes are allowed, even if k is increased by ties.)"
prob,"If this is true, the proportion of the votes for the winning class are returned as attribute prob."
use.all,"controls handling of ties. If true, all distances equal to the kth largest are included. If false, a random selection of distances equal to the kth is chosen to use exactly k neighbours."
